In [51]:
#### 1065 Calculation #####
import pandas as pd
import numpy as np
import scipy as sc
import scipy.optimize as opt
import xlsxwriter

data_raw = pd.read_csv("Test_data.csv", header=2,low_memory=False)
data_raw = data_raw.convert_objects(convert_numeric=True)
data_un = pd.DataFrame()

## Prepare constant Values
bs = 0.00821722713286476 # Has to be calculated
RFPF = 0.0133
CH4_RF = 1.1
Tchiller = 7
Pamb = 98.0918852
Pchiller = Pamb + 23.75
xCO2intdry = 0.000375
xCO2dildry = 0.000375
xTHC_THC_FID_init = 1

# Choose whether COL or COH
species_raw = [col for col in list(data_raw) if col.startswith('E_CO')]
if 'E_COHD2' in species_raw:
    CO = 'E_COHD2'
    CO_flag = 1 # CO-High
else:
    CO = 'E_COLD2'
    CO_flag = 0 # CO-Low

print("Initialization finished!")

Initialization finished!


In [52]:
##### First Level of Calculation #####
data_un["E_CO2D2"] = data_raw.E_CO2D2*10000
data_un[CO] = data_raw.get(CO)*10000
data_un["E_NOXD2"] = data_raw.E_NOXD2
data_un["E_THCW2"] = data_raw.E_THCW2
data_un["xCH4wet"] = data_raw.E_CH4W2/1000000
data_un["C_FRAIRWS"] = data_raw.C_FRAIRWS
data_un["BARO Press"] = data_raw.P_INLET*100
data_un["T_INLET"] = data_raw.T_INLET
data_un["M_RELHUM"] = data_raw.M_RELHUM
data_un["mfuel"] = data_raw.M_FRFUEL*1000/3600
print("First Level of Calculation finished!")

First Level of Calculation finished!


In [53]:
##### Second Level of Calculation #####
data_un["xCO2meas"] = data_un.E_CO2D2/1000000
data_un["xCOmeas"] = data_un.get(CO)/1000000
data_un["xNOxmeas"] = data_un.E_NOXD2/1000000
data_un["xTHCmeas"] = data_un.E_THCW2/1000000
data_un["Molar Flow Wet"] = data_un.C_FRAIRWS*1000/3600
data_un["maf_kgh (wet)"] = data_un.C_FRAIRWS
data_un["pH2O @ inlet"] = 10**(10.79574*(1-(273.16/(273.15+data_un.T_INLET)))-5.028*np.log10((273.15+data_un.T_INLET)/273.16)+0.000150475*(1-10**(-8.2969*(((273.15+data_un.T_INLET)/273.16)-1)))+0.00042873*(10**(4.76955*(1-(273.16/(273.15+data_un.T_INLET))))-1)-0.2138602)  ## Reference: CFR 1065.564
data_un["xH2O"] = data_un.M_RELHUM*data_un.get("pH2O @ inlet")/(data_un.get("BARO Press")*100) ## Reference: CFR 1065.564
data_un["xTHC[THC_FID]cor"] = data_un.E_THCW2-xTHC_THC_FID_init ## Reference: CFR 1065.660
print("Second Level of Calculation finished!")

Second Level of Calculation finished!


In [54]:
##### Third Level of Calculation #####
data_un.E_THCW2 = data_un.get("xTHC[THC_FID]cor") ## Reference: CFR 1065.660
data_un["xNO2meas"] = data_un.xNOxmeas*0
data_un["xNOmeas"] = data_un.xNOxmeas*1
data_un["xTHCwet"] = data_un.xTHCmeas
data_un["Intake Air flow"] = data_un.get("Molar Flow Wet")/28.96
data_un["Mmix"] = 28.96559*(1-data_un.xH2O)+18.01528*data_un.xH2O
data_un["xH2Oint"] = data_un.xH2O
data_un["xH2Odil"] = data_un.xH2O

# Constant Channels
c = np.zeros((len(data_un),1))
c.fill(3.5)
data_un["xH2Ogas"] = c
c.fill(13.875389)
data_un["Molar Mass of HC Emissions"] = c
c.fill(28.0101)
data_un["Molar Mass of CO Emissions"] = c
c.fill(46.0055)
data_un["Molar Mass of NOx Emissions (as NO2)"] = c
c.fill(44.0095)
data_un["Molar Mass of CO2 Emissions"] = c
c.fill(13.875389)
data_un["Molar Mass of NMHC Emissions"] = c
c.fill(0.866)
data_un["Wc"] = c
c.fill(1.84)
data_un['alpha'] = c
c.fill(0)
data_un['beta'] = c
data_un['gamma'] = c
data_un['delta'] = c
c = None
print("Third Level of Calculation finished!")

Third Level of Calculation finished!


In [55]:
##### Fourth Level of Calculation #####
data_un["xNMHCwet"] = (data_un.xTHCwet-data_un.xCH4wet*CH4_RF)/(1-RFPF*CH4_RF) ## Reference: CFR 1065.660
data_un["nint (Intake Air Flow)"] = data_un.get("Molar Flow Wet")/data_un.Mmix
data_un["xH2Ointdry"] = data_un.xH2Oint/(1-data_un.xH2Oint)
data_un["xH2Odildry"] = data_un.xH2Odil/(1-data_un.xH2Odil)
print("Fourth Level of Calculation finished!")

Fourth Level of Calculation finished!


In [56]:
##### Fifth Level of Calculation #####
data_un["xCO2int"] = xCO2intdry/(1+data_un.xH2Ointdry)
data_un["xO2int"] = ((0.20982-xCO2intdry)/(1+data_un.xH2Ointdry)) 
data_un["xCO2dil"] = xCO2dildry/(1+data_un.xH2Odildry)
print("Fifth Level of Calculation finished!")

Fifth Level of Calculation finished!


In [ ]:
##### Circle Calculation #####
def f(variables):
    # Channels in Formulas
    #A = xdil/exh -- G
    #B = xH2Oexh -- H
    #C = xCcombdry -- I
    #D = xH2Oexhdry -- J
    #E = xdil/exhdry -- K
    #F = xint/exhdry -- L
    #G = xraw/exhdry -- M
    #H = xH2OCOmeas -- AC
    #I = xH2OTHCmeas -- AD
    #J = xH2ONOxmeas -- AE
    #K = xH2ONO2meas -- AF
    #L = xH2OCO2meas -- AH
    #M = xCOdry -- AI
    #N = xTHCdry -- AJ
    #O = xNOxdry -- AK
    #P = xNO2dry -- AL
    #Q = xCO2dry -- AN
    #R = xH2dry -- AO 
    
    # Unknown Variables
    (A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R) = variables
        
    # Known Variables
    S = data_un.xCO2dil[i]
    T = data_un.xCO2int[i]
    U = data_un.alpha[i]
    V = data_un.xH2Odil[i]
    W = data_un.xH2Oint[i]
    X = data_un.xO2int[i]
    Y = data_un.beta[i]
    Z = data_un.gamma[i]
    AA = data_un.delta[i]
    AB = data_un.xCOmeas[i]
    AC = data_un.xTHCmeas[i]
    AD = data_un.xNOxmeas[i]
    AE = data_un.xNO2meas[i]
    AF = data_un.xH2Ogas[i]
    AG = data_un.xCO2meas[i]
    
    # Equations to solve
    eq1 = 1-(G/(1+D))-A
    eq2 = D/(1+D)-B
    eq3 = Q+(M)+(N)-(S*E)-(T*F)-C
    eq4 = ((U/2)*(C-N))+V*E+W*F-R-D 
    eq5 = A/(1-B)-E
    eq6 = (1/(2*X))*(((U/2)-Y+2+(2*Z))*(C-N)-(M-O-(2*P)+R))-F
    eq7 = 0.5*(((U/2)+Y+AA)*(C-N)+((2*N)+M-P+R))+F-G
    eq8 = AB/(1-H)-M ## Reference: CFR 1065.659
    eq10 = AC/(1-I)-N ## Reference: CFR 1065.659
    eq11 = B-I
    eq12 = AD/(1-J)-O ## Reference: CFR 1065.659
    eq14 = AE/(1-K)-P ## Reference: CFR 1065.659
    eq16 = (M*(D-V*E))/(AF*(Q-S*E))-R ## Reference: CFR 1065.659
    eq17 = AG/(1-L)-Q ## Reference: CFR 1065.659
    
    if mode==0:
        eq9 = bs-H    
        eq13 = bs-J
        eq15 = bs-K
        eq18 = bs-L
    else:
        eq9 = B-H    
        eq13 = B-J
        eq15 = B-K
        eq18 = B-L
    
    return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14,eq15,eq16,eq17,eq18]

i = 0
result_list = np.zeros((len(data_un),18))

while i < len(data_un)-1:
    i +=1  
    g = 0.5 # First guess for iteration start

    solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
    
    if solution[2]<bs:
        mode = 1
        solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
        mode = 0
    result_list[:][i] = solution

iteration = pd.DataFrame(result_list,columns =['xdil/exh','xH2Oexh','xCcombdry','xH2Oexhdry','xdil/exhdry','xint/exhdry','xraw/exhdry','xH2OCOmeas','xH2OTHCmeas','xH2ONOxmeas','xH2ONO2meas','xH2OCO2meas','xCOdry','xTHCdry','xNOxdry','xNO2dry','xCO2dry','xH2dry'])
data_un = pd.concat([data_un,iteration],axis=1)

# Clear Variables
mode = None
result_list = None
g = None
solution = None
iteration = None
print("Circle Calculation finished!")
## 05/06/2015 - Anton lang, Lang@FEV.com

In [58]:
##### Calculation of remaining values #####
data_un["xH2ONOmeas"] = data_un.xH2ONO2meas
data_un["xNOdry"] = data_un.xNOmeas/(1-data_un.xH2ONOmeas) ## Reference: CFR 1065.659
data_un["xCO2dry"] = data_un.xCO2meas/(1-data_un.xH2OCO2meas) ## Reference: CFR 1065.659
data_un["xH2dry"] = (data_un.xCOdry*(data_un.xH2Oexhdry-data_un.xH2Odil*data_un.get("xdil/exhdry")))/(data_un.xH2Ogas*(data_un.xCO2dry-data_un.xCO2dil*data_un.get("xdil/exhdry")))
data_un["xCOwet"] = data_un.xCOmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2OCOmeas))
data_un['xNOxwet'] = data_un.xNOxmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONOxmeas))
data_un["xNO2wet"] = data_un.xNO2meas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONO2meas))
data_un["xNOwet"] = data_un.xNOmeas*((1-data_un.xH2Oexh)/(1-data_un.xH2ONOmeas))
data_un["xCO2wet"] = data_un.xCO2meas*((1-data_un.xH2Oexh)/(1-data_un.xH2OCO2meas))
data_un["xNOxcorrwet"] = data_un.xNOxwet*(18.84 *data_un.xH2O + 0.68094) ## Reference: CFR 1065.670

data_un["nexh"] = data_un.get("nint (Intake Air Flow)")/(1+((data_un.get("xint/exhdry")-data_un.get("xraw/exhdry"))/(1+data_un.xH2Oexhdry)))
data_un["Mass_THC"] = data_un.xTHCwet*data_un.nexh*data_un.get("Molar Mass of HC Emissions")
data_un["Mass_CO"] = data_un.xCOwet*data_un.nexh*data_un.get("Molar Mass of CO Emissions")
data_un["Mass_NOx"] = data_un.xNOxcorrwet*data_un.nexh*data_un.get("Molar Mass of NOx Emissions (as NO2)")
data_un["Mass_CO2"] = data_un.xCO2wet*data_un.nexh*data_un.get("Molar Mass of CO2 Emissions")
data_un["Mass_NMHC"] = data_un.xNMHCwet*data_un.nexh*data_un.get("Molar Mass of NMHC Emissions")

# Emissions in total
sum_THC_un = data_un.Mass_THC.sum(skipna = True)
sum_CO_un = data_un.Mass_CO.sum(skipna = True)
sum_NOx_un = data_un.Mass_NOx.sum(skipna=True)
sum_CO2_un = data_un.Mass_CO2.sum(skipna=True)
sum_NMHC_un = data_un.Mass_NMHC.sum(skipna=True)
array_sum_un = [sum_CO2_un,sum_CO_un,sum_NOx_un,sum_THC_un,sum_NMHC_un]
print("Uncorrected Calculation finished!")

Uncorrected Calculation finished!


In [75]:
###############################################################
##################### Drift Correction ########################
###############################################################

In [78]:
##### Load Pre/Post - Files #####
data_pre = pd.read_csv("Pre_zero_span.csv", low_memory=False)
data_pre = data_pre.convert_objects(convert_numeric=True)
data_post = pd.read_csv("Post_zero_span.csv", low_memory=False)
data_post = data_post.convert_objects(convert_numeric=True)
data_cor = pd.DataFrame()
data_raw2 = data_raw

species_pre = [col for col in list(data_pre) if col.startswith('E_')]

zero_span = pd.DataFrame(data=np.zeros([5,5]),index=['PreZero','PreSpan','PostZero','PostSpan','Chosen'],columns=['E_CO2D2',CO,'E_NOXD2','E_THCW2','E_CH4W2'])
print('Data Load finished!')

Data Load finished!


In [60]:
##### Pre-File #####
zero_data = data_pre.loc[59, species_pre[0]:species_pre[-1]]
span_data = data_pre.loc[121, species_pre[0]:species_pre[-1]]

## Zero -  Data ##
zero_span.E_CO2D2['PreZero'] = abs(zero_data.E_CO2D2+0) * 10000
zero_span[CO]['PreZero'] = abs(zero_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PreZero'] = abs(zero_data.E_NOXD2+0)
zero_span.E_THCW2['PreZero'] = abs(zero_data.E_THCW2+0)

## Span - Data ##
zero_span.E_CO2D2['PreSpan'] = abs(data_pre.E_CO2D2.iloc[-1]+0) * 10000
zero_span[CO]['PreSpan'] = abs(span_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PreSpan'] = abs(span_data.E_NOXD2+0)
zero_span.E_THCW2['PreSpan'] = abs(span_data.E_THCW2+0)
zero_span.E_CH4W2['PreSpan'] = abs(data_pre.E_THCW2.iloc[-1]+0)
print('Pre-Zero/Span is finished!')

Pre-Zero/Span is finished!


In [203]:
b = data_pre.loc[1:60, species_pre[1]]

for i in range(1,len(b)):
    if b[i]< 0.3:
        b = b.drop(i)
        
b.mean()

0.34890909090909089

In [209]:
zero_span = pd.DataFrame(data=np.zeros([5,5]),index=['PreZero','PreSpan','PostZero','PostSpan','Chosen'],columns=species)
zero_span
name = {'Zero':'PreZero','Span':'PreSpan'}
name['Zero']

'PreZero'

In [190]:
        Species = ['CO2','CO','NOX','THC','CH4'] #these are the species in the array but with different names

        TypeTime = ['Zero_start','Zero_end','Span_start','Span_end']
        SpecTime = [0,59,60,119] # Time window for Zero and Span Data
        time_window = pd.DataFrame(data = np.ones([4,5]), columns=Species, index=TypeTime)
        for time_point, spec_time in zip(type_time,SpecTime):
            time_window.loc[time_point,:] = spec_time
            
        time_window[Species[0]]['Span_start'] = 120
        time_window[Species[0]]['Span_end'] = 179
        time_window[Species[4]]['Span_start'] = 120
        time_window[Species[4]]['Span_end'] = 179
        
time_window['CO2'][1]

59.0

In [61]:
##### Post-File #####
zero_data = data_post.loc[59, species_pre[0]:species_pre[-1]]
span_data = data_post.loc[120, species_pre[0]:species_pre[-1]]

## Zero -  Data ##
zero_span.E_CO2D2['PostZero'] = abs(zero_data.E_CO2D2+0) * 10000
zero_span[CO]['PostZero'] = abs(zero_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PostZero'] = abs(zero_data.E_NOXD2+0)
zero_span.E_THCW2['PostZero'] = abs(zero_data.E_THCW2+0)
zero_span.E_CH4W2['PostZero'] = abs(zero_data.E_THCW2+0)

## Span - Data ##
zero_span.E_CO2D2['PostSpan'] = abs(data_post.E_CO2D2.iloc[-1]+0) * 10000
zero_span[CO]['PostSpan'] = abs(span_data.get(CO)+0) * 10000
zero_span.E_NOXD2['PostSpan'] = abs(span_data.E_NOXD2+0)
zero_span.E_THCW2['PostSpan'] = abs(span_data.E_THCW2+0)
zero_span.E_CH4W2['PostSpan'] = abs(data_post.E_THCW2.iloc[-1]+0)

## Chosen Values - From Database (Tanks)
zero_span.E_CO2D2['Chosen'] = 191000
zero_span[CO]['Chosen'] = 58700
zero_span.E_NOXD2['Chosen'] = 4005
zero_span.E_THCW2['Chosen'] = 9900
zero_span.E_CH4W2['Chosen'] = 2510
print('Post-Zero/Span is finished!')

Post-Zero/Span is finished!


In [62]:
zero_span.E_NOXD2['PreZero']


0.5

In [79]:
######### TEST #############
zero_data = data_pre.loc[59,species_pre[0]:species_pre[-1]]
span_data = data_pre.loc[120,species_pre[0]:species_pre[-1]]
## Zero -  Data ##
zero_span.E_CO2D2['PreZero'] = 0
zero_span[CO]['PreZero'] = 0
zero_span.E_NOXD2['PreZero'] = 0.412
zero_span.E_THCW2['PreZero'] = 0.254

## Span - Data ##
zero_span.E_CO2D2['PreSpan'] = 190030
zero_span[CO]['PreSpan'] = 58700
zero_span.E_NOXD2['PreSpan'] = 4003.95
zero_span.E_THCW2['PreSpan'] = 9899.89
zero_span.E_CH4W2['PreSpan'] = 2654.17

zero_data = data_post.loc[59,species_pre[0]:species_pre[-1]]
span_data = data_post.loc[120,species_pre[0]:species_pre[-1]]
## Zero -  Data ##
zero_span.E_CO2D2['PostZero'] = 50
zero_span[CO]['PostZero'] = 0
zero_span.E_NOXD2['PostZero'] = 0.914
zero_span.E_THCW2['PostZero'] = 0.074
zero_span.E_CH4W2['PostZero'] = 0.074

## Span - Data ##
zero_span.E_CO2D2['PostSpan'] = 190400
zero_span[CO]['PostSpan'] = 58850
zero_span.E_NOXD2['PostSpan'] = 4000.1
zero_span.E_THCW2['PostSpan'] = 9890.16
zero_span.E_CH4W2['PostSpan'] = 2651.84

In [63]:
## Correct Raw-Emissions ##
data_raw2['E_CO2D2'] = zero_span.E_CO2D2['Chosen']*((2*data_un.E_CO2D2)-(zero_span.E_CO2D2['PreZero']+zero_span.E_CO2D2['PostZero']))/((zero_span.E_CO2D2['PreSpan']+zero_span.E_CO2D2['PostSpan'])-(zero_span.E_CO2D2['PreZero']+zero_span.E_CO2D2['PostZero']))
data_raw2[CO] = zero_span[CO]['Chosen']*((2*data_un.get(CO))-(zero_span[CO]['PreZero']+zero_span[CO]['PostZero']))/((zero_span[CO]['PreSpan']+zero_span[CO]['PostSpan'])-(zero_span[CO]['PreZero']+zero_span[CO]['PostZero']))
data_raw2['E_NOXD2'] = zero_span.E_NOXD2['Chosen']*((2*data_un.E_NOXD2)-(zero_span.E_NOXD2['PreZero']+zero_span.E_NOXD2['PostZero']))/((zero_span.E_NOXD2['PreSpan']+zero_span.E_NOXD2['PostSpan'])-(zero_span.E_NOXD2['PreZero']+zero_span.E_NOXD2['PostZero']))
data_raw2['E_THCW2'] = zero_span.E_THCW2['Chosen']*((2*data_un.E_THCW2)-(zero_span.E_THCW2['PreZero']+zero_span.E_THCW2['PostZero']))/((zero_span.E_THCW2['PreSpan']+zero_span.E_THCW2['PostSpan'])-(zero_span.E_THCW2['PreZero']+zero_span.E_THCW2['PostZero']))
data_raw2['E_CH4W2'] = zero_span.E_CH4W2['Chosen']*((2*data_raw.E_CH4W2)-(zero_span.E_CH4W2['PreZero']+zero_span.E_CH4W2['PostZero']))/((zero_span.E_CH4W2['PreSpan']+zero_span.E_CH4W2['PostSpan'])-(zero_span.E_CH4W2['PreZero']+zero_span.E_CH4W2['PostZero']))
print('Raw Emissions drift-corrected!')

Raw Emissions drift-corrected!


In [64]:
##### First Level of Calculation #####
data_cor["E_CO2D2"] = data_raw2.E_CO2D2
data_cor[CO] = data_raw2.get(CO)
data_cor["E_NOXD2"] = data_raw2.E_NOXD2
data_cor["E_THCW2"] = data_raw2.E_THCW2
data_cor["xCH4wet"] = data_raw2.E_CH4W2/1000000 # ppm --> mol/mol
data_cor["C_FRAIRWS"] = data_raw2.C_FRAIRWS
data_cor["BARO Press"] = data_raw2.P_INLET*100 # in kPa
data_cor["T_INLET"] = data_raw2.T_INLET
data_cor["M_RELHUM"] = data_raw2.M_RELHUM
data_cor["mfuel"] = data_raw2.M_FRFUEL*1000/3600 # kg/h --> g/s
print("First Level of Calculation finished!")

First Level of Calculation finished!


In [65]:
##### Second Level of Calculation #####
data_cor["xCO2meas"] = data_cor.E_CO2D2/1000000 # ppm --> mol/mol
data_cor["xCOmeas"] = data_cor.get(CO)/1000000 # ppm --> mol/mol
data_cor["xNOxmeas"] = data_cor.E_NOXD2/1000000 # ppm --> mol/mol
data_cor["xTHCmeas"] = data_cor.E_THCW2/1000000 # ppm --> mol/mol
data_cor["Molar Flow Wet"] = data_cor.C_FRAIRWS*1000/3600 # kg/h --> g/s
data_cor["maf_kgh (wet)"] = data_cor.C_FRAIRWS
data_cor["pH2O @ inlet"] = 10**(10.79574*(1-(273.16/(273.15+data_cor.T_INLET)))-5.028*np.log10((273.15+data_cor.T_INLET)/273.16)+0.000150475*(1-10**(-8.2969*(((273.15+data_cor.T_INLET)/273.16)-1)))+0.00042873*(10**(4.76955*(1-(273.16/(273.15+data_cor.T_INLET))))-1)-0.2138602)
data_cor["xH2O"] = data_cor.M_RELHUM*data_cor.get("pH2O @ inlet")/(data_cor.get("BARO Press")*100)
data_cor["xTHC[THC_FID]cor"] = data_cor.E_THCW2-xTHC_THC_FID_init
print("Second Level of Calculation finished!")

Second Level of Calculation finished!


In [66]:
##### Third Level of Calculation #####
data_cor["xNO2meas"] = data_cor.xNOxmeas*0
data_cor["xNOmeas"] = data_cor.xNOxmeas*1
data_cor["xTHCwet"] = data_cor.xTHCmeas
data_cor["Intake Air flow"] = data_cor.get("Molar Flow Wet")/28.96 # g/sec --> mol/sec
data_cor["Mmix"] = 28.96559*(1-data_cor.xH2O)+18.01528*data_cor.xH2O
data_cor["xH2Oint"] = data_cor.xH2O
data_cor["xH2Odil"] = data_cor.xH2O

# Constant Channels
c = np.zeros((len(data_cor),1))
c.fill(3.5)
data_cor["xH2Ogas"] = c
c.fill(13.875389)
data_cor["Molar Mass of HC Emissions"] = c
c.fill(28.0101)
data_cor["Molar Mass of CO Emissions"] = c
c.fill(46.0055)
data_cor["Molar Mass of NOx Emissions (as NO2)"] = c
c.fill(44.0095)
data_cor["Molar Mass of CO2 Emissions"] = c
c.fill(13.875389)
data_cor["Molar Mass of NMHC Emissions"] = c

# Fuel Specs
c.fill(0.866)
data_cor["Wc"] = c
c.fill(1.84)
data_cor['alpha'] = c
c.fill(0)
data_cor['beta'] = c
data_cor['gamma'] = c
data_cor['delta'] = c
c = None
print("Third Level of Calculation finished!")

Third Level of Calculation finished!


In [67]:
##### Fourth Level of Calculation #####
data_cor["xNMHCwet"] = (data_cor.xTHCwet-data_cor.xCH4wet*CH4_RF)/(1-RFPF*CH4_RF)
data_cor["nint (Intake Air Flow)"] = data_cor.get("Molar Flow Wet")/data_cor.Mmix
data_cor["xH2Ointdry"] = data_cor.xH2Oint/(1-data_cor.xH2Oint)
data_cor["xH2Odildry"] = data_cor.xH2Odil/(1-data_cor.xH2Odil)
print("Fourth Level of Calculation finished!")

Fourth Level of Calculation finished!


In [68]:
##### Fifth Level of Calculation #####
data_cor["xCO2int"] = xCO2intdry/(1+data_cor.xH2Ointdry)
data_cor["xO2int"] = ((0.20982-xCO2intdry)/(1+data_cor.xH2Ointdry)) 
data_cor["xCO2dil"] = xCO2dildry/(1+data_cor.xH2Odildry)
print("Fifth Level of Calculation finished!")

Fifth Level of Calculation finished!


In [69]:
##### Circle Calculation #####
def f(variables):    
    # Unknown Variables
    (A,B,C,D,E,F,G,H,I,J,K,L,M,N,O,P,Q,R) = variables
        
    # Known Variables
    S = data_cor.xCO2dil[i]
    T = data_cor.xCO2int[i]
    U = data_cor.alpha[i]
    V = data_cor.xH2Odil[i]
    W = data_cor.xH2Oint[i]
    X = data_cor.xO2int[i]
    Y = data_cor.beta[i]
    Z = data_cor.gamma[i]
    AA = data_cor.delta[i]
    AB = data_cor.xCOmeas[i]
    AC = data_cor.xTHCmeas[i]
    AD = data_cor.xNOxmeas[i]
    AE = data_cor.xNO2meas[i]
    AF = data_cor.xH2Ogas[i]
    AG = data_cor.xCO2meas[i]
    
    # Equations to solve
    eq1 = 1-(G/(1+D))-A
    eq2 = D/(1+D)-B
    eq3 = Q+(M)+(N)-(S*E)-(T*F)-C
    eq4 = ((U/2)*(C-N))+V*E+W*F-R-D 
    eq5 = A/(1-B)-E
    eq6 = (1/(2*X))*(((U/2)-Y+2+(2*Z))*(C-N)-(M-O-(2*P)+R))-F
    eq7 = 0.5*(((U/2)+Y+AA)*(C-N)+((2*N)+M-P+R))+F-G
    eq8 = AB/(1-H)-M    
    eq10 = AC/(1-I)-N
    eq11 = B-I
    eq12 = AD/(1-J)-O    
    eq14 = AE/(1-K)-P    
    eq16 = (M*(D-V*E))/(AF*(Q-S*E))-R
    eq17 = AG/(1-L)-Q
    
    if mode==0:
        eq9 = bs-H    
        eq13 = bs-J
        eq15 = bs-K
        eq18 = bs-L
    else:
        eq9 = B-H    
        eq13 = B-J
        eq15 = B-K
        eq18 = B-L
    
    return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11,eq12,eq13,eq14,eq15,eq16,eq17,eq18]

i = 0
mode = 0
result_list = np.zeros((len(data_cor),18))

while i < len(data_cor)-1:
    i +=1  
    g = 0.5 # First guess for iteration start

    solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
    
    if solution[2]<bs:
        mode = 1
        solution = opt.fsolve(f,(g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g,g))
        mode = 0
    result_list[:][i] = solution

iteration = pd.DataFrame(result_list,columns =['xdil/exh','xH2Oexh','xCcombdry','xH2Oexhdry','xdil/exhdry','xint/exhdry','xraw/exhdry','xH2OCOmeas','xH2OTHCmeas','xH2ONOxmeas','xH2ONO2meas','xH2OCO2meas','xCOdry','xTHCdry','xNOxdry','xNO2dry','xCO2dry','xH2dry'])
data_cor = pd.concat([data_cor,iteration],axis=1)

# Clear Variables
mode = None
result_list = None
g = None
solution = None
iteration = None
print("Circle Calculation finished!")

Circle Calculation finished!


In [72]:
##### Calculation of remaining values #####
data_cor["xH2ONOmeas"] = data_cor.xH2ONO2meas
data_cor["xNOdry"] = data_cor.xNOmeas/(1-data_cor.xH2ONOmeas)
data_cor["xCO2dry"] = data_cor.xCO2meas/(1-data_cor.xH2OCO2meas)
data_cor["xH2dry"] = (data_cor.xCOdry*(data_cor.xH2Oexhdry-data_cor.xH2Odil*data_cor.get("xdil/exhdry")))/(data_cor.xH2Ogas*(data_cor.xCO2dry-data_cor.xCO2dil*data_cor.get("xdil/exhdry")))
data_cor["xCOwet"] = data_cor.xCOmeas*((1-data_cor.xH2Oexh)/(1-data_cor.xH2OCOmeas))
data_cor['xNOxwet'] = data_cor.xNOxmeas*((1-data_cor.xH2Oexh)/(1-data_cor.xH2ONOxmeas))
data_cor["xNO2wet"] = data_cor.xNO2meas*((1-data_cor.xH2Oexh)/(1-data_cor.xH2ONO2meas))
data_cor["xNOwet"] = data_cor.xNOmeas*((1-data_cor.xH2Oexh)/(1-data_cor.xH2ONOmeas))
data_cor["xCO2wet"] = data_cor.xCO2meas*((1-data_cor.xH2Oexh)/(1-data_cor.xH2OCO2meas))
data_cor["xNOxcorrwet"] = data_cor.xNOxwet*(18.84 *data_cor.xH2O + 0.68094)

data_cor["nexh"] = data_cor.get("nint (Intake Air Flow)")/(1+((data_cor.get("xint/exhdry")-data_cor.get("xraw/exhdry"))/(1+data_cor.xH2Oexhdry)))
data_cor["Mass_THC"] = data_cor.xTHCwet*data_cor.nexh*data_cor.get("Molar Mass of HC Emissions")
data_cor["Mass_CO"] = data_cor.xCOwet*data_cor.nexh*data_cor.get("Molar Mass of CO Emissions")
data_cor["Mass_NOx"] = data_cor.xNOxcorrwet*data_cor.nexh*data_cor.get("Molar Mass of NOx Emissions (as NO2)")
data_cor["Mass_CO2"] = data_cor.xCO2wet*data_cor.nexh*data_cor.get("Molar Mass of CO2 Emissions")
data_cor["Mass_NMHC"] = data_cor.xNMHCwet*data_cor.nexh*data_cor.get("Molar Mass of NMHC Emissions")

# Emissions in total
sum_THC_cor = data_cor.Mass_THC.sum(skipna = True)
sum_CO_cor = data_cor.Mass_CO.sum(skipna = True)
sum_NOx_cor = data_cor.Mass_NOx.sum(skipna=True)
sum_CO2_cor = data_cor.Mass_CO2.sum(skipna=True)
sum_NMHC_cor = data_cor.Mass_NMHC.sum(skipna=True)
array_sum_cor = [sum_CO2_cor,sum_CO_cor,sum_NOx_cor,sum_THC_cor,sum_NMHC_cor]

# Emissions in total (Negative Values removed)
for i in range(1,len(data_cor)):
    if data_cor.Mass_THC[i]<0:
        data_cor.Mass_THC[i] = 0
    if data_cor.Mass_CO.ix[i]<0:
        data_cor.Mass_CO.ix[i] = 0
    if data_cor.Mass_NOx.ix[i]<0:
        data_cor.Mass_NOx.ix[i] = 0
    if data_cor.Mass_CO2.ix[i]<0:        
        data_cor.Mass_CO2.ix[i] = 0
    if data_cor.Mass_NMHC.ix[i]<0:
        data_cor.Mass_NMHC.ix[i] = 0
    if data_raw.U_BPOW[i]<0:
        data_raw.U_BPOW[i] = 0
U_BPOW_Factor = data_raw.U_BPOW.sum(skipna=True)/(3600*0.746)
sum_THC_cor_won = data_cor.Mass_THC.sum(skipna=True)
sum_CO_cor_won = data_cor.Mass_CO.sum(skipna = True)
sum_NOx_cor_won = data_cor.Mass_NOx.sum(skipna=True)
sum_CO2_cor_won = data_cor.Mass_CO2.sum(skipna=True)
sum_NMHC_cor_won = data_cor.Mass_NMHC.sum(skipna=True)
array_sum_cor_won = [sum_CO2_cor_won,sum_CO_cor_won,sum_NOx_cor_won,sum_THC_cor_won,sum_NMHC_cor_won]
print("Drift-corrected Calculation finished!")

Drift-corrected Calculation finished!


/usr/local/lib/python3.3/dist-packages/IPython/kernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [70]:
##### Final Report ######
final = pd.DataFrame()
final['Species'] = ['CO2','CO','NOx','THC','NMHC']
final['Units'] = ['g/ghphr','g/ghphr','g/ghphr','g/ghphr','g/ghphr']
CO2 = sum_CO2_cor_won/U_BPOW_Factor
CO = sum_CO_cor_won/U_BPOW_Factor
NOx = sum_NOx_cor_won/U_BPOW_Factor
THC = sum_THC_cor_won/U_BPOW_Factor
NMHC = sum_NMHC_cor_won/U_BPOW_Factor
final['Cold Start Em.'] = [CO2,CO,NOx,THC,NMHC]
final['Hot Start Em.'] = [753.071,8.288,0.071,0.104,0.057]
total = final.get('Cold Start Em.')/7 + final.get('Hot Start Em.')*6/7
final['Total'] = total
final['DF'] = [1,1.53,3.268,1,1.961]
final_emi = total*final.DF
final['FINAL Em.'] = final_emi
final['Regulation'] = ['-',14.4,0.2,'-',0.14]

# Preparation of Excel-File
file = xlsxwriter.Workbook('Final.xlsx')
sheet = file.add_worksheet('Emissions_Calculations')
bold = file.add_format({'bold': True})
red = file.add_format({'font_color': 'red','border':1,'bold':1,'fg_color':'#FFA62F'})
dark_grey = file.add_format({'fg_color':'#696969','bold':1,'border': 1})
bright_grey = file.add_format({'fg_color':'#A9A9A9', 'bold':1,'border': 1})
border = file.add_format({'border':1})
merge_format = file.add_format({'bold': 1,'border': 1,'align': 'center','valign': 'vcenter','fg_color': '#C71585','font_color':'white'})
merge_format2 = file.add_format({'bold': 1,'border': 1,'align': 'center','valign': 'vcenter','fg_color': '#A9A9A9'})

# Final Emissions
sheet.merge_range('A1:H1', 'Final Emissions', merge_format)
sheet.write_row('A2',final.columns.values,dark_grey)
sheet.write_column('A3',final.Species,bright_grey)
sheet.write_column('B3',final.Units,bright_grey)
sheet.write_column('C3',np.round(final.get('Cold Start Em.'),3),border)
sheet.write_column('D3',np.round(final.get('Hot Start Em.'),3),border)
sheet.write_column('E3',np.round(final.get('Total'),3),border)
sheet.write_column('F3',final.get('DF'),border)
sheet.write_column('H3',final.get('Regulation'),bright_grey)

# Emissions Mass - corrected without negative values
sheet.merge_range('J1:K1', 'Emissions Mass', merge_format2)
sheet.write('J2','Cold Start',bright_grey)
sheet.write('K2','Hot Start',bright_grey)
sheet.write_column('J3',array_sum_cor_won,border)

final.Regulation = final.Regulation.convert_objects(convert_numeric=True)
for i in range(1,len(final)+1):
    value = round(final.get('FINAL Em.')[i-1],3)
    if value > final.Regulation[i-1]:
        string = 'G' + str(i+2)
        sheet.write(string,value,red)
    else:
        string = 'G' + str(i+2)
        sheet.write(string,value,border)        
        
# Drift corrected emissions
drift = pd.DataFrame()
drift['Species'] = ['CO2','CO','NOx','THC','NMHC']
drift['Units'] = ['g/ghphr','g/ghphr','g/ghphr','g/ghphr','g/ghphr']
CO2 = sum_CO2_cor/U_BPOW_Factor
CO = sum_CO_cor/U_BPOW_Factor
NOx = sum_NOx_cor/U_BPOW_Factor
THC = sum_THC_cor/U_BPOW_Factor
NMHC = sum_NMHC_cor/U_BPOW_Factor
drift['Cold Start Em.'] = [CO2,CO,NOx,THC,NMHC]
drift['Hot Start Em.'] = [753.071,8.288,0.07,0.104,0.057]
total = drift.get('Cold Start Em.')/7 + drift.get('Hot Start Em.')*6/7
drift['Total'] = total
drift['DF'] = [1,1.53,3.268,1,1.961]
drift_emi = total*drift.DF
drift['FINAL Em.'] = drift_emi
drift['Regulation'] = ['-',14.4,0.2,'-',0.14]
sheet.merge_range('A10:H10', 'Drift corrected Emissions', merge_format)
sheet.write_row('A11',drift.columns.values,dark_grey)
sheet.write_column('A12',drift.Species,bright_grey)
sheet.write_column('B12',drift.Units,bright_grey)
sheet.write_column('C12',np.round(drift.get('Cold Start Em.'),3),border)
sheet.write_column('D12',np.round(drift.get('Hot Start Em.'),3),border)
sheet.write_column('E12',np.round(drift.get('Total'),3),border)
sheet.write_column('F12',drift.get('DF'),border)
sheet.write_column('H12',drift.get('Regulation'),bright_grey)
drift.Regulation = drift.Regulation.convert_objects(convert_numeric=True)

# Emissions Mass - corrected
sheet.merge_range('J10:K10', 'Emissions Mass', merge_format2)
sheet.write('J11','Cold Start',bright_grey)
sheet.write('K11','Hot Start',bright_grey)
sheet.write_column('J12',array_sum_cor,border)

for i in range(1,len(drift)+1):
    value = round(drift.get('FINAL Em.')[i-1],3)
    if value > drift.Regulation[i-1]:
        string = 'G' + str(i+11)
        sheet.write(string,value,red)
    else:
        string = 'G' + str(i+11)
        sheet.write(string,value,border)

# Uncorrected emissions
un = pd.DataFrame()
un['Species'] = ['CO2','CO','NOx','THC','NMHC']
un['Units'] = ['g/ghphr','g/ghphr','g/ghphr','g/ghphr','g/ghphr']
CO2 = sum_CO2_un/U_BPOW_Factor
CO = sum_CO_un/U_BPOW_Factor
NOx = sum_NOx_un/U_BPOW_Factor
THC = sum_THC_un/U_BPOW_Factor
NMHC = sum_NMHC_un/U_BPOW_Factor
un['Cold Start Em.'] = [CO2,CO,NOx,THC,NMHC]
un['Hot Start Em.'] = [750.206,8.3,0.073,0.104,0.054]
total = un.get('Cold Start Em.')/7 + un.get('Hot Start Em.')*6/7
un['Total'] = total
un['DF'] = [1,1.53,3.268,1,1.961]
un_emi = total*un.DF
un['FINAL Em.'] = un_emi
un['Regulation'] = ['-',14.4,0.2,'-',0.14]
sheet.merge_range('A19:H19', 'Drift uncorrected emissions', merge_format)
sheet.write_row('A20',un.columns.values,dark_grey)
sheet.write_column('A21',un.Species,bright_grey)
sheet.write_column('B21',un.Units,bright_grey)
sheet.write_column('C21',np.round(un.get('Cold Start Em.'),3),border)
sheet.write_column('D21',np.round(un.get('Hot Start Em.'),3),border)
sheet.write_column('E21',np.round(un.get('Total'),3),border)
sheet.write_column('F21',un.get('DF'),border)
sheet.write_column('H21',un.get('Regulation'),bright_grey)
un.Regulation = un.Regulation.convert_objects(convert_numeric=True)

# Emissions Mass - uncorrected
sheet.merge_range('J19:K19', 'Emissions Mass', merge_format2)
sheet.write('J20','Cold Start',bright_grey)
sheet.write('K20','Hot Start',bright_grey)
sheet.write_column('J21',array_sum_un,border)

for i in range(1,len(un)+1):
    value = round(un.get('FINAL Em.')[i-1],3)
    if value > drift.Regulation[i-1]:
        string = 'G' + str(i+20)
        sheet.write(string,value,red)
    else:
        string = 'G' + str(i+20)
        sheet.write(string,value,border)

file.close()
print('1065 Calculation finished! Report is saved!')

NameError: name 'sum_CO2_cor_won' is not defined